In [1]:
import pandas as pd
import numpy as np
import statistics as stat
from datetime import datetime

In [2]:
dailies=pd.read_excel(r'C:\Users\wells\OneDrive\Desktop\大學部\dailies.xlsx')
sleepLevel=pd.read_excel(r'C:\Users\wells\OneDrive\Desktop\大學部\sleepLevelsMap.xlsx')
dailies1=pd.read_excel(r'C:\Users\wells\OneDrive\Desktop\大學部\dailies.xlsx')
sleepLevel1=pd.read_excel(r'C:\Users\wells\OneDrive\Desktop\大學部\碩士\sleeps_levelsmap.xlsx')
dailies=pd.concat([dailies,dailies1],ignore_index=True)
sleepLevel=pd.concat([sleepLevel,sleepLevel1],ignore_index=True)

In [3]:

sleepLeveldrop=['date']

In [4]:
dailiesdrop=['summaryId','activityType','date','startTimeOffsetInSeconds','steps','distanceInMeters','floorsClimbed','stepsGoal',
             'intensityDurationGoalInSeconds','floorsClimbedGoal','stressDurationInSeconds','restStressDurationInSeconds',
            'activityStressDurationInSeconds','lowStressDurationInSeconds','mediumStressDurationInSeconds',
             'highStressDurationInSeconds','stressQualifier','這段持續時間活動秒數','中等強度持續秒數','強烈強度持續秒數',
            '最低心跳','最高心跳','每分鐘平均心跳','休息時間每分鐘平均心跳','最大壓力等級']

In [5]:

sleepLevel=sleepLevel.drop(sleepLeveldrop,axis=1)
dailies=dailies.drop(dailiesdrop,axis=1)

In [6]:
dailies['總消耗卡路里']=dailies['活動消耗卡路里']+dailies['基礎代謝率']

In [7]:
dailies['日期時間']=dailies.開始時間.apply(datetime.fromtimestamp)
sleepLevel['開始']=sleepLevel.開始時間.apply(datetime.fromtimestamp)
sleepLevel['結束']=sleepLevel.結束時間.apply(datetime.fromtimestamp)
sleepLevel['持續時間']=sleepLevel['結束時間']-sleepLevel['開始時間']

In [8]:
dailies=dailies.drop_duplicates(subset=None, keep='first', inplace=False)

In [9]:
dailies=dailies.reset_index(drop=True)

In [10]:
sleepLevel=sleepLevel.drop_duplicates(subset=None, keep='first', inplace=False)

In [11]:
sleepLevel=sleepLevel.reset_index(drop=True)

In [12]:
dailies=dailies.sort_values(by=['id','日期時間']).reset_index(drop=True)
dailies=dailies.drop_duplicates(['id','日期時間'],keep='last')
dailies=dailies.reset_index(drop=True)

In [13]:
ids=list(set(dailies['id']))
dailies_new=pd.DataFrame({'id':[],
                         '活動消耗卡路里':[],
                         '基礎代謝率':[],
                         '開始時間':[],
                         '總消耗卡路里':[],
                         '日期時間':[],
                         '總卡路里標準化':[],
                         '平均壓力正規化':[]})
for idn in ids:
    daily=dailies[dailies['id'].isin([idn])]
    cal=max(daily['基礎代謝率'])
    daily['活動消耗卡路里']=cal/daily['基礎代謝率']*daily['活動消耗卡路里']
    daily['基礎代謝率']=cal
    daily['總消耗卡路里']=daily['活動消耗卡路里']+daily['基礎代謝率']
    daily['總卡路里標準化']=(daily['總消耗卡路里']-stat.mean(daily['總消耗卡路里']))/stat.stdev(daily['總消耗卡路里'])
    daily['平均壓力等級']=cal/daily['基礎代謝率']*daily['平均壓力等級']
    daily['平均壓力正規化']=(daily['平均壓力等級']-stat.mean(daily['平均壓力等級']))/stat.stdev(daily['平均壓力等級'])
    dailies_new=pd.concat([dailies_new,daily],ignore_index=True)

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [14]:
dailies_new=dailies_new.reset_index(drop=True)
dailies_new=dailies_new.drop(['開始時間','持續時間','活動消耗卡路里','基礎代謝率'],axis=1)

In [15]:
firsttime=1553400000
light=[]
deep=[]
rem=[]
all_time=[]
all_id=[]
for idn in ids:
    sleepLevelid=sleepLevel[sleepLevel['id'].isin([idn])]
    firsttime=1553400000
    while  firsttime<1558756801:
        time=[]
        for each_time in range(firsttime,firsttime+86400,60):
            time.append(each_time)
        sleepLevelidtime=sleepLevelid[sleepLevelid['開始時間'].isin(time)]
        all_id.append(idn)
        all_time.append(firsttime)
        light.append(sum(sleepLevelidtime[sleepLevelidtime['睡眠程度'].isin(['light'])]['持續時間']))
        deep.append(sum(sleepLevelidtime[sleepLevelidtime['睡眠程度'].isin(['deep'])]['持續時間']))
        rem.append(sum(sleepLevelidtime[sleepLevelidtime['睡眠程度'].isin(['rem'])]['持續時間']))
        firsttime+=86400
sleep_new=pd.DataFrame({'id':all_id,
                       '時間':all_time,
                       '快速動眼':rem,
                       '淺層':light,
                       '深層':deep})

In [16]:
sleep_new['日期']=sleep_new.時間.apply(datetime.fromtimestamp)
sleep_new=sleep_new.drop('時間',axis=1)

In [17]:
sleep_new=sleep_new.drop(sleep_new.reset_index()[sleep_new.reset_index()['快速動眼'].isin([0])]['index'])
sleep_new=sleep_new.reset_index(drop=True)

In [18]:
sleep_new['深度睡眠比例']=sleep_new['深層']/(sleep_new['快速動眼']+sleep_new['淺層']+sleep_new['深層'])

In [19]:
ids=list(set(sleep_new['id']))

In [20]:
sleep_add=pd.DataFrame({'深層比例正規化':[]})
for idn in ids:
    sleep_new1=sleep_new[sleep_new['id'].isin([idn])]
    sleep_new1['深層比例正規化']=(sleep_new1['深度睡眠比例']-stat.mean(sleep_new1['深度睡眠比例']))/stat.stdev(sleep_new['深度睡眠比例'])
    sleep_add=pd.concat([sleep_add,sleep_new1],ignore_index=True)

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [43]:
sleepspo=pd.DataFrame({
    
})

In [48]:
spo2=pd.read_excel(r'C:\Users\wells\OneDrive\Desktop\大學部\sleeps_spo2.xlsx')
spo21=pd.read_excel(r'C:\Users\wells\OneDrive\Desktop\大學部\碩士\sleeps_SleepSpo2.xlsx')
spo2=pd.concat([spo2,spo21],ignore_index=True)

In [37]:
summary=list(set(spo2['summaryId']))

In [51]:
spo2=spo2.drop(['date','Unnamed: 0'],axis=1)

In [49]:
spo2=spo2.drop_duplicates(subset=None, keep='first', inplace=False)

In [50]:
spo2

,Unnamed: 0,id,summaryId,date,timeseconds,sleeps_HeartRate
0,0,e,x2d07022-5ca0c72c-9894,407,12900,90
1,1,e,x2d07022-5ca0c72c-9894,407,12960,87
2,2,e,x2d07022-5ca0c72c-9894,407,13020,87
3,3,e,x2d07022-5ca0c72c-9894,407,13080,94
4,4,e,x2d07022-5ca0c72c-9894,407,13140,95
5,5,e,x2d07022-5ca0c72c-9894,407,13200,92
6,6,e,x2d07022-5ca0c72c-9894,407,13260,92
7,7,e,x2d07022-5ca0c72c-9894,407,13320,91
8,8,e,x2d07022-5ca0c72c-9894,407,13380,90
9,9,e,x2d07022-5ca0c72c-9894,407,13440,92


In [45]:
for each_sid in list(spo2['summaryId']):
    sleep_=sleepLevel[sleepLevel['summaryId'].isin([each_sid])]
    spo_=spo2[spo2['summaryId'].isin([each_sid])]
    spo_['時間']=sum(sleep_['開始時間'])/len(sleep_['開始時間'])
    spo_['快速動眼']=sum(sleep_[sleep_['睡眠程度'].isin(['rem'])]['持續時間'])
    spo_['淺層']=sum(sleep_[sleep_['睡眠程度'].isin(['light'])]['持續時間'])
    spo_['深層']=sum(sleep_[sleep_['睡眠程度'].isin(['deep'])]['持續時間'])
    sleepspo=pd.concat([sleepspo,spo_],ignore_index=True)

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

KeyboardInterrupt: 

In [ ]:
sleepspo=sleepspo.drop(sleepspo.reset_index()[sleepspo.reset_index()['快速動眼'].isin([0])]['index'])
sleepspo=sleepspo.reset_index(drop=True)

In [ ]:
sleepspo=sleepspo.drop(sleepspo.reset_index()[sleepspo.reset_index()['深層'].isin([0])]['index'])
sleepspo=sleepspo.reset_index(drop=True)

In [ ]:
sleepspo['深度睡眠比例']=sleepspo['深層']/(sleepspo['快速動眼']+sleepspo['淺層']+sleepspo['深層'])

In [ ]:
sleepspo['日期時間']=sleepspo.時間.apply(datetime.fromtimestamp)

In [ ]:
sleepspo.drop(['時間'],axis=1)
sleepspo['深度睡眠比例']=sleepspo['深層']/(sleepspo['快速動眼']+sleepspo['淺層']+sleepspo['深層'])

In [ ]:
dailies_new.to_excel(r'C:\Users\wells\Desktop\dailie.xlsx',index=False)
sleep_add.to_excel(r'C:\Users\wells\Desktop\sleeplevel.xlsx',index=False)
sleepspo.to_excel(r'C:\Users\wells\Desktop\sleep+spo.xlsx',index=False)

In [51]:
dailyheart=pd.concat([dailyheart_1,dailyheart_2,dailyheart_3],ignore_index=True)

In [52]:
dailyheart=dailyheart.drop('summaryId',axis=1)

In [53]:
dailyheart=dailyheart.rename(columns={'date':'日期','timeseconds':'時間秒','HeartRate':'心跳'})

In [59]:
dailyheart_e=dailyheart[dailyheart['id'].isin(['e'])]